In [8]:
from copyreg import constructor
from evogym import is_connected, has_actuator, get_full_connectivity, hashable
from evogym.envs import StepsUp, WalkingFlat
import struct
from typing import List, Tuple
import numpy as np
import dill
from src.graph import Graph
from src.evolution_strategies import tournament_selection
from src.population import Population
from src.arg_parser import parse_args
from pygifsicle import optimize
import imageio
import test

In [9]:
FILE_PATH = "results/no_fit_share_01.pkl"
GENERATION = "99"
ENV = "StepsUp" # "[WalkingFlat, StepsUp]"

In [10]:
structure_pop: Population = None
with open(FILE_PATH, "rb") as file:
    structure_pop, robot_dict = dill.load(file)
    
def get_observation(env):
    a = env.get_vel_com_obs("robot")
    b = env.get_pos_com_obs("robot")
    c = env.get_floor_obs("robot", ["ground"], 5)
    return np.concatenate((a, b, c))

# def get_observation(env):
#     a = env.get_vel_com_obs("robot")
#     b = env.get_pos_com_obs("robot")
#     return np.concatenate((a, b))
    
def generate_robot(g: Graph, structure):
    robot = np.zeros(structure)
    for i in range(structure[0]):
        for j in range(structure[1]):
            input = (i - (structure[0] // 2),
                     j - (structure[1] // 2))
            graph_out = g.operate(input)
            node = np.argmax(graph_out)
            robot[i][j] = node
    return robot

def get_best_controller(robot, controller_population, n_steps):
    robot = generate_robot(design, (5,5))    
    connections = get_full_connectivity(robot)

    
    env = StepsUp(body=robot, connections=connections)
    
    fitness = []
    cont = 0
    for controller in controller_population.indvs:
        env.reset()
        reward = test.calculate_reward(env, controller, 500)
        fitness.append((cont, reward, controller.id))
        cont += 1
        
    fitness.sort(key=lambda x: (x[1], x[0]))
    
    return fitness

ModuleNotFoundError: No module named 'population'

In [9]:
structure_pop.indvs.sort(key=lambda x: (x.original_fit*-1, x.id))
cont = 0
print("\t ID \t  Fitness \t Species")
for design in structure_pop.indvs:
    if design.original_fit > -999:
       
        print(f"{cont}:\t {design.id}\t| {design.original_fit:.4f}\t| {design.species_id}")
    cont+=1

	 ID 	  Fitness 	 Species
0:	 238841	| 0.0000	| 987
1:	 377193	| 0.0000	| 923
2:	 384408	| 0.0000	| 972
3:	 386844	| 0.0000	| 976
4:	 386846	| 0.0000	| 990
5:	 386848	| 0.0000	| 991
6:	 386849	| 0.0000	| 986
7:	 386851	| 0.0000	| 978
8:	 386853	| 0.0000	| 963
9:	 386855	| 0.0000	| 986
10:	 386857	| 0.0000	| 978
11:	 386859	| 0.0000	| 972
12:	 386861	| 0.0000	| 985
13:	 386863	| 0.0000	| 978
14:	 386865	| 0.0000	| 976
15:	 386867	| 0.0000	| 979
16:	 386869	| 0.0000	| 952
17:	 386871	| 0.0000	| 986
18:	 386873	| 0.0000	| 978
19:	 386875	| 0.0000	| 991
20:	 386876	| 0.0000	| 963
21:	 386878	| 0.0000	| 972
22:	 386879	| 0.0000	| 990
23:	 386881	| 0.0000	| 992
24:	 386883	| 0.0000	| 976
25:	 386885	| 0.0000	| 993
26:	 386887	| 0.0000	| 972
27:	 386889	| 0.0000	| 923
28:	 386891	| 0.0000	| 976
29:	 386893	| 0.0000	| 986


In [10]:
STEPS = 600
DESIGN_ID = 0
SAVE_GIF = False

In [12]:
design =structure_pop.indvs[DESIGN_ID]

robot = generate_robot(design, (5,5))
# robot = unhash("0002200024000440044400444", (5,5))
connections = get_full_connectivity(robot)
print(robot)

if ENV == "StepsUp":
    env = StepsUp(body=robot, connections=connections)
else:
    env = WalkingFlat(body=robot, connections=connections)


env.reset()
if not SAVE_GIF:
    env.render('screen')

controller_pop = robot_dict[hashable(robot)].population
controller_pop.indvs.sort(key=lambda x: (x.original_fit*-1, x.id))
controller = controller_pop.indvs[0]
print(controller.id)

reward = 0
actuators = env.get_actuator_indices("robot")
imgs = []
for _ in range(STEPS):
    obs = get_observation(env)

    action_by_actuator = controller.operate(obs, reset_fit=False)
    action = [action_by_actuator[i] for i in actuators]
#     action = np.clip(action, .6, 1.6)
    _, r, done, _ = env.step(np.array(action))
    
    if SAVE_GIF:
        imgs.append(env.render(mode='img'))
    else:
        env.render('screen')
    reward += r

    if done:
        print("AAA")
        break

        
print(f'\ntotal reward: {round(reward, 5)}\n')
env.close()

if SAVE_GIF:
    print(len(imgs))
    imageio.mimsave(f'{ENV}_gen_{GENERATION}_c{CONTROLLER_ID}_d_{DESIGN_ID}.gif', imgs, duration=(1/60))

[[3. 3. 1. 1. 1.]
 [2. 2. 1. 1. 1.]
 [4. 4. 4. 0. 0.]
 [3. 3. 4. 0. 0.]
 [3. 3. 4. 0. 0.]]
301468

total reward: 4.83391



In [7]:
if SAVE_GIF:
    optimize(f'{ENV}_gen_{GENERATION}_c{CONTROLLER_ID}_d_{DESIGN_ID}.gif')

In [26]:
save_top = 20
ordered_robots = {k: v for k, v in sorted(robot_dict.items(), key=lambda item: -1*item[1].best_fit)}
best_robots = {}
cont = 0
for k, v in ordered_robots.items():
    if cont >= save_top:
        break
    best_robots[k] = v
    cont+=1
for indv in structure_pop.indvs:
    robot = generate_robot(indv, (5,5))
    h = hashable(robot) 
    if h not in best_robots and h in robot_dict:
        best_robots[h] = robot_dict[h]
print(len(best_robots))

29
